# Open 2020: Winner Predictor
## Diego Chinellato - 867637
## Giorgia Campardo - 867928
### Web Intelligence Course

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel('2019+.xlsx')

In [3]:
print()

In [4]:
df.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,LBW,LBL,PSW,PSL,SJW,SJL,MaxW,MaxL,AvgW,AvgL
0,1,Brisbane,Brisbane International,2018-12-31,ATP250,Outdoor,Hard,1st Round,3,Dimitrov G.,...,NaN,NaN,1.36,3.37,NaN,NaN,1.42,3.60,1.35,3.18
1,1,Brisbane,Brisbane International,2018-12-31,ATP250,Outdoor,Hard,1st Round,3,Raonic M.,...,NaN,NaN,1.23,4.68,NaN,NaN,1.27,4.84,1.22,4.26
2,1,Brisbane,Brisbane International,2018-12-31,ATP250,Outdoor,Hard,1st Round,3,Kecmanovic M.,...,NaN,NaN,1.67,2.32,NaN,NaN,1.71,2.40,1.63,2.28
3,1,Brisbane,Brisbane International,2018-12-31,ATP250,Outdoor,Hard,1st Round,3,Millman J.,...,NaN,NaN,1.41,3.13,NaN,NaN,1.45,3.20,1.40,2.95
4,1,Brisbane,Brisbane International,2018-12-31,ATP250,Outdoor,Hard,1st Round,3,Uchiyama Y.,...,NaN,NaN,2.73,1.51,NaN,NaN,3.26,1.53,2.69,1.47


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23634 entries, 0 to 23633
Data columns (total 42 columns):
ATP           23634 non-null int64
Location      23634 non-null object
Tournament    23634 non-null object
Date          23634 non-null datetime64[ns]
Series        23634 non-null object
Court         23634 non-null object
Surface       23634 non-null object
Round         23634 non-null object
Best of       23634 non-null int64
Winner        23634 non-null object
Loser         23634 non-null object
WRank         23624 non-null float64
LRank         23586 non-null float64
WPts          23626 non-null float64
LPts          23587 non-null float64
W1            23483 non-null float64
L1            23485 non-null float64
W2            23260 non-null float64
L2            23260 non-null float64
W3            11173 non-null float64
L3            11173 non-null float64
W4            2200 non-null float64
L4            2200 non-null float64
W5            816 non-null float64
L5          

In [6]:
df.describe()

,ATP,Best of,WRank,LRank,WPts,LPts,W1,L1,W2,L2,...,LBW,LBL,PSW,PSL,SJW,SJL,MaxW,MaxL,AvgW,AvgL
count,23634.000000,23634.000000,23624.000000,23586.000000,23626.000000,23587.000000,23483.000000,23485.000000,23260.000000,23260.000000,...,20172.000000,20183.000000,23513.000000,23513.000000,10314.000000,10321.000000,23609.000000,23609.000000,23609.000000,23609.000000
mean,33.036896,3.386139,56.754614,87.187908,1988.829002,1127.641497,5.806583,4.105770,5.784007,3.952193,...,1.817708,3.494537,1.927335,4.180454,1.801868,3.618600,1.996621,8.105400,1.837950,3.558816
std,18.040228,0.789431,70.940513,109.198180,2477.960179,1294.122302,1.225518,1.834737,1.246084,1.857456,...,1.065951,3.207252,1.382640,5.489767,1.020090,3.432987,1.593275,389.900091,1.092432,3.254419
min,1.000000,3.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.010000,1.000000,1.010000,1.010000,1.010000,1.010000,1.010000
25%,19.000000,3.000000,16.000000,34.000000,693.000000,557.000000,6.000000,3.000000,6.000000,3.000000,...,1.250000,1.730000,1.270000,1.790000,1.220000,1.730000,1.300000,1.840000,1.250000,1.740000
50%,33.000000,3.000000,40.000000,62.000000,1060.000000,788.000000,6.000000,4.000000,6.000000,4.000000,...,1.500000,2.500000,1.550000,2.640000,1.500000,2.630000,1.580000,2.750000,1.510000,2.530000
75%,49.000000,3.000000,74.000000,100.000000,2030.000000,1195.000000,6.000000,6.000000,6.000000,6.000000,...,2.000000,4.000000,2.140000,4.240000,2.000000,4.000000,2.210000,4.460000,2.070000,3.920000
max,67.000000,5.000000,1890.000000,2159.000000,16950.000000,16950.000000,7.000000,7.000000,7.000000,7.000000,...,26.000000,51.000000,46.000000,121.000000,17.000000,81.000000,76.000000,42586.000000,23.450000,36.440000


In [7]:
def preprocess_data(df, missing_values="drop", drop_first=False):
    """
    Processes raw data and returns a tuple (X, Y) where X is the cleaned dataset and Y is the array of labels.
    """
    # Generate labels
    Y = np.concatenate([np.ones(df.shape[0], dtype=int), np.zeros(df.shape[0], dtype=int)])
    # Drop unuseful columns
    X = df.drop(columns=['ATP', 'Location', 'Tournament', 'Date', 'Comment'])
    
    # Deal with missing valus
    
    if missing_values == 'drop':
        X = X.drop(columns=[])
    elif missing_values == 'univariate':
        from sklearn.impute import SimpleImputer
        # go from here
        imp = SimpleImputer(missing_values=np.nan, strategy='mean')
        imp.fit(X)  
        pass
    elif missing_values == 'multivariate':
        pass
    
    # Convert ordinal features to int (higher value means more important)
    series = ['ATP250', 'ATP500', 'Masters 1000', 'Masters Cup', 'Grand Slam']
    series2int = {s: i for i, s in enumerate(series)}
    rounds2int = {'1st Round': 0,
                  '2nd Round': 1,
                  '3rd Round': 2,
                  '4th Round': 3,
                  'Round Robin': 4,
                  'Quarterfinals': 5,
                  'Semifinals': 6,
                  'The Final': 7,
                 }
    X = X.replace({'Round': rounds2int, 'Series': series2int})
    
    # One hot encode categorical features into binary features
    X = pd.get_dummies(X, prefix=['Court_', 'Surface_'], columns=['Court', 'Surface'], drop_first=drop_first)
    
    # Convert players to numeric ?
    players = set(X['Winner']) | set(X['Loser'])
    players_to_id = {}
    for i, player in enumerate(players):
        players_to_id[player] = i
    X = X.replace({'Winner': players_to_id, 'Loser': players_to_id})
    X = X.astype({'Winner':int, 'Loser':int})
    
    X = X.rename(columns={'Winner':'1st Player', 'Loser':'2nd Player'})
    return X, Y

In [8]:
data, labels = preprocess_data(df)

In [9]:
data.head()
data.loc[:,'W1':'W5'].head()

,W1,L1,W2,L2,W3,L3,W4,L4,W5
0,6.0,3.0,6.0,4.0,NaN,NaN,NaN,NaN,NaN
1,6.0,0.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN
2,6.0,3.0,6.0,1.0,NaN,NaN,NaN,NaN,NaN
3,7.0,6.0,6.0,7.0,6.0,0.0,NaN,NaN,NaN
4,6.0,4.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN


In [10]:
print(data['L5'].unique())
data.info()

[nan  2.  4.  0.  6.  3.  1.  9.  5. 10.  7. 20.  8. 11. 14. 16. 12. 15.
 13. 17. 24.]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23634 entries, 0 to 23633
Data columns (total 40 columns):
Series            23634 non-null int64
Round             23634 non-null int64
Best of           23634 non-null int64
1st Player        23634 non-null int32
2nd Player        23634 non-null int32
WRank             23624 non-null float64
LRank             23586 non-null float64
WPts              23626 non-null float64
LPts              23587 non-null float64
W1                23483 non-null float64
L1                23485 non-null float64
W2                23260 non-null float64
L2                23260 non-null float64
W3                11173 non-null float64
L3                11173 non-null float64
W4                2200 non-null float64
L4                2200 non-null float64
W5                816 non-null float64
L5                816 non-null float64
Wsets             23482 non-null float64


In [12]:
def dowload_ao2020_data():
    """
    To get the data for the tournament? if we need to scrape these are the links of the matches
    https://ausopen.com/tournament-schedule
    https://ausopen.com/schedule#!8071
    """